In [ ]:
import pandas as pd
import duckdb
import plotly.express as px
from pathlib import Path

# Configurar pandas para mostrar mais colunas se necessário
pd.set_option('display.max_columns', 100)

In [ ]:
# O caminho é relativo à localização do seu notebook.
# Se o notebook está na raiz do projeto, este caminho deve funcionar.
DUCKDB_FILE_PATH = Path("dados/sngpc_analytics.duckdb")
TABLE_NAME = "prescricoes"

# Conectar ao banco de dados em modo read-only
try:
    con = duckdb.connect(database=str(DUCKDB_FILE_PATH), read_only=True)
    print("Conexão com o DuckDB estabelecida com sucesso!")
except Exception as e:
    print(f"Erro ao conectar ao DuckDB: {e}")

In [ ]:
# Executa uma consulta e carrega o resultado em um DataFrame pandas
# Use .fetchdf() para obter um DataFrame pandas
df_amostra = con.execute(f"SELECT * FROM {TABLE_NAME} LIMIT 10;").fetchdf()

# Em um notebook, simplesmente ter o nome do DataFrame na última linha de uma célula
# irá exibi-lo de forma bem formatada.
df_amostra

In [ ]:
query_tipo_receituario = f"""
    SELECT
        tipo_receituario,
        SUM(quantidade_vendida) as total_vendido,
        COUNT(*) as numero_prescricoes
    FROM {TABLE_NAME}
    WHERE ano = 2020
    GROUP BY tipo_receituario
    ORDER BY total_vendido DESC;
"""

df_resultado_receituario = con.execute(query_tipo_receituario).fetchdf()

df_resultado_receituario

In [ ]:
# Criar um gráfico de barras com os resultados da nossa query ad-hoc
fig = px.bar(
    df_resultado_receituario,
    x='tipo_receituario',
    y='total_vendido',
    title='Total de Quantidade Vendida por Tipo de Receituário em 2020',
    labels={'total_vendido': 'Quantidade Total Vendida', 'tipo_receituario': 'Tipo de Receituário'}
)

# Para exibir o gráfico no notebook
fig.show()

In [ ]:
# Fechar a conexão
con.close()
print("Conexão com o DuckDB fechada.")

In [4]:
import pandas as pd
import duckdb
from pathlib import Path
import sys

# --- 1. Configuração e Conexão ---
print("--- INICIANDO AUDITORIA PÓS-ETL NO BANCO DE DADOS ---")

# Adiciona a pasta raiz ao caminho para encontrar os utilitários
project_root = Path.cwd()
if not (project_root / 'src').is_dir():
    # Se o notebook estiver em um subdiretório, ajuste o caminho
    project_root = Path(__file__).resolve().parent.parent

sys.path.append(str(project_root))

try:
    from src.utils.database_utils import DUCKDB_FILE_PATH, TABLE_NAME, TABLE_MAPEAMENTO

    # Conecta ao banco de dados em modo de apenas leitura
    conexao = duckdb.connect(database=str(DUCKDB_FILE_PATH), read_only=True)
    print(f"Conexão estabelecida com o banco: {DUCKDB_FILE_PATH}")

    # --- 2. Inspeção do Schema das Tabelas ---
    print("\n[PASSO 1] Verificando a estrutura (schema) das tabelas...")

    print("\nEstrutura da Tabela 'prescricoes':")
    schema_prescricoes = conexao.execute(f"DESCRIBE {TABLE_NAME};").fetchdf()
    print(schema_prescricoes.to_string())

    print("\nEstrutura da Tabela 'mapeamento_controlados':")
    schema_mapeamento = conexao.execute(f"DESCRIBE {TABLE_MAPEAMENTO};").fetchdf()
    print(schema_mapeamento.to_string())

    # --- 3. Extração de Amostras por Capital ---
    print("\n[PASSO 2] Extraindo uma amostra de 5 registros para cada capital...")
    
    # Esta consulta SQL usa uma "window function" para numerar as linhas dentro de cada 
    # grupo de 'nome_municipio' e depois seleciona as 5 primeiras de cada grupo.
    query_amostra_por_capital = f"""
    WITH AmostraNumerada AS (
        SELECT
            *,
            ROW_NUMBER() OVER(PARTITION BY nome_municipio ORDER BY data DESC) as rn
        FROM {TABLE_NAME}
    )
    SELECT * FROM AmostraNumerada
    WHERE rn <= 5
    ORDER BY nome_municipio, data DESC;
    """
    
    df_amostra = conexao.execute(query_amostra_por_capital).fetchdf()
    
    if df_amostra.empty:
        print("\nNenhuma amostra pôde ser gerada. A tabela 'prescricoes' pode estar vazia.")
    else:
        print(f"-> Amostra extraída com sucesso! Total de {len(df_amostra)} registros.")
        print("\n[PASSO 3] Exibindo amostra de dados por capital:")
        
        # Configuração do pandas para exibir todas as colunas
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 200)

        # Exibe o DataFrame diretamente no notebook
        # O Jupyter renderiza isso como uma tabela formatada.
        display(df_amostra)

except Exception as e:
    print(f"\nOcorreu um erro durante a auditoria: {e}")
finally:
    if 'conexao' in locals() and conexao:
        conexao.close()
        print("\nConexão com DuckDB fechada.")

print("\n--- AUDITORIA CONCLUÍDA ---")

--- INICIANDO AUDITORIA PÓS-ETL NO BANCO DE DADOS ---
Conexão estabelecida com o banco: /Users/rondiseba/Desktop/CP_Rondi/Especialização IA Dados /app_SNGPC/dados/sngpc_analytics.duckdb

[PASSO 1] Verificando a estrutura (schema) das tabelas...

Estrutura da Tabela 'prescricoes':
                   column_name column_type null   key default extra
0                          ano     INTEGER  YES  None    None  None
1                          mes     INTEGER  YES  None    None  None
2                         data        DATE  YES  None    None  None
3                     sigla_uf     VARCHAR  YES  None    None  None
4              principio_ativo     VARCHAR  YES  None    None  None
5       descricao_apresentacao     VARCHAR  YES  None    None  None
6                      dosagem     VARCHAR  YES  None    None  None
7           forma_farmaceutica     VARCHAR  YES  None    None  None
8           quantidade_vendida      DOUBLE  YES  None    None  None
9                        cid10     VA

,ano,mes,data,sigla_uf,principio_ativo,descricao_apresentacao,dosagem,forma_farmaceutica,quantidade_vendida,cid10,sexo,idade,anvisa_lista,idade_modificada_flag,quantidade_modificada_flag,faixa_etaria,periodo_valido_controlado,nome_municipio,conselho_prescritor,rn
0,2020,12,2020-12-01,SE,MOXIFLOXACINO + FOSFATO DISSÃDICO DE DEXAMETA...,5 MG/ML + 1 MG/ML SOL OFT CT FR PLAS OPC GOT X...,5 MG/ML,Solução Oral,1.0,Não Informado,Masculino,49,None,0,0,Adulto (25-59),<NA>,Aracaju,CRM,1
1,2020,12,2020-12-01,SE,MOXIFLOXACINO + FOSFATO DISSÃDICO DE DEXAMETA...,5 MG/ML + 1 MG/ML SOL OFT CT FR PLAS OPC GOT X...,5 MG/ML,Solução Oral,1.0,Não Informado,Masculino,40,None,0,0,Adulto (25-59),<NA>,Aracaju,CRM,2
2,2020,12,2020-12-01,SE,CIPROFLOXACINO,500 MG COM REV CT BL AL PLAS TRANS X 14,500 MG,Comprimido Revestido,1.0,Não Informado,Feminino,55,ANTIMICROBIANO,0,0,Adulto (25-59),True,Aracaju,CRM,3
3,2020,12,2020-12-01,SE,CEFADROXILA,500 MG CAP DURA CT ENV AL PLAS X 8Â Â,500 MG,Cápsula,2.0,Não Informado,Feminino,25,ANTIMICROBIANO,0,0,Adulto (25-59),True,Aracaju,CRO,4
4,2020,12,2020-12-01,SE,NORFLOXACINO,400MG COM REV CT BL AL/PLAS TRANS X 14,400MG,Comprimido Revestido,2.0,Não Informado,Masculino,50,ANTIMICROBIANO,0,0,Adulto (25-59),True,Aracaju,CRM,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,2020,12,2020-12-01,ES,AZITROMICINA DI-HIDRATADA,500 MG COM REV CT BL AL PLAS PVC OPC X 5,500 MG,Comprimido Revestido,6.0,Não Informado,Masculino,27,ANTIMICROBIANO,0,0,Adulto (25-59),True,Vitória,CRM,1
131,2020,12,2020-12-01,ES,AMOXICILINA,500 MG CAP DURA CT BL AL PLAS PVDC TRANS X 21,500 MG,Cápsula,1.0,Não Informado,Feminino,35,ANTIMICROBIANO,0,0,Adulto (25-59),True,Vitória,CRM,2
132,2020,12,2020-12-01,ES,SULFAMETOXAZOL + TRIMETOPRIMA,800 MG + 160 MG COM CT BL AL PLAS TRANS X 10,800 MG,Não Especificada,6.0,Não Informado,Feminino,67,ANTIMICROBIANO,0,1,Idoso (65+),True,Vitória,CRM,3
133,2020,12,2020-12-01,ES,AZITROMICINA DI-HIDRATADA,500 MG COM REV CT BL AL PLAS OPC X 5,500 MG,Comprimido Revestido,5.0,Não Informado,Feminino,83,ANTIMICROBIANO,0,0,Idoso (65+),True,Vitória,CRM,4



Conexão com DuckDB fechada.

--- AUDITORIA CONCLUÍDA ---
